In [1]:
"""
Insert all the feature names
"""

!nrnivmodl modfiles
all_features = {'StepNeg': set(['time_constant', u'time_constant', u'voltage_deflection_begin',
                               'voltage_deflection_begin'
                               ]),            
            
                'StepPos':set(['mean_frequency', 'AP_amplitude'
                              
                              ])}

/home/iavarone/BBP/fens2016/SimulationTutorials/FENS2016/scoring
modfiles/CaDynamics.mod modfiles/Ca_HVA.mod modfiles/Ca_LVA.mod modfiles/Ih.mod modfiles/Im.mod modfiles/Im_v2.mod modfiles/K_P.mod modfiles/K_T.mod modfiles/Kd.mod modfiles/Kv2like.mod modfiles/Kv3_1.mod modfiles/NaTa.mod modfiles/NaTs.mod modfiles/NaV.mod modfiles/Nap.mod modfiles/SK.mod
CaDynamics.mod Ca_HVA.mod Ca_LVA.mod Ih.mod Im.mod Im_v2.mod K_P.mod K_T.mod Kd.mod Kv2like.mod Kv3_1.mod NaTa.mod NaTs.mod NaV.mod Nap.mod SK.mod
"/home/iavarone/neuron/nrn/share/nrn/libtool" --tag=CC --mode=compile mpicc -DHAVE_CONFIG_H  -I. -I.. -I"/home/iavarone/neuron/nrn/include/nrn" -I"/home/iavarone/neuron/nrn/x86_64/lib"      -g -O2 -c -o mod_func.lo mod_func.c
libtool: compile:  mpicc -DHAVE_CONFIG_H -I. -I.. -I/home/iavarone/neuron/nrn/include/nrn -I/home/iavarone/neuron/nrn/x86_64/lib -g -O2 -c mod_func.c  -fPIC -DPIC -o .libs/mod_func.o
"/home/iavarone/neuron/nrn/share/nrn/libtool" --tag=CC --mode=link mpicc -module  -g -O2

In [2]:
%load_ext autoreload
%autoreload

from allensdk.api.queries.biophysical_api import BiophysicalApi
%matplotlib notebook
import matplotlib.pyplot as plt
import collections

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy
import json

import efel

import cell_model
import cell_evaluator

# Load the .nwb file containing the voltage traces
from allensdk.core.nwb_data_set import NwbDataSet
nwb_data = NwbDataSet('../ABI_model/data/475049288.nwb')

# Store data in a dictionary step_name:sweep_num
steps_dict = collections.OrderedDict(sorted({'StepNeg':23,'StepPos':36}.items(), key = lambda x:x[0]))

def get_data(dataset,sweeps_dict):
    # Correct for the liquid junction potential
    junction_potential = -14
    data = collections.OrderedDict()
    for step, sweep in sweeps_dict.items():
        sweep_data = {}
        # Get sweep from NWB dataset
        sweep = dataset.get_sweep(sweep)

        # Get stimulus from the sweep
        # stimulus = sweep['stimulus']

        # Sampling rate is in Hz
        sampling_rate = sweep['sampling_rate']
        
        # Start/stop indices that exclude the experimental test pulse (if applicable)
        index_range = sweep['index_range']
        
        # Response is a numpy array in mV
        sweep_data['V'] = sweep['response'][index_range[0]:index_range[1]]*1000+junction_potential    
        
        # Define some time points in milliseconds (i.e., convert to absolute time)
        sweep_data['T'] = numpy.arange(0,(len(sweep_data['V'])/sampling_rate),1./sampling_rate)*1000
        data[step] = sweep_data
    return data

data_dict = get_data(nwb_data, steps_dict)

# Extract features
    
import efel
def get_features(data, feature_names):
    
    traces_pas = []
    traces_act = []
    for step_name, step_traces in data.items():
        trace = {}
        trace['T'] = data[step_name]['T']
        trace['V'] = data[step_name]['V']
        trace['stim_start'] = [270]
        trace['stim_end'] = [1270]
        trace['name'] = step_name
    
        if 'Pos' in step_name:
            traces_act.append(trace)       
        else:
            traces_pas.append(trace)
            
    features_values = efel.getMeanFeatureValues(traces_pas, feature_names['StepNeg'])
    features_values_act = efel.getMeanFeatureValues(traces_act, feature_names['StepPos']) 
    
    features_values.extend(features_values_act)
    return features_values

efel_features = get_features(data_dict, all_features)

# Write features mean values and std in a json file
features_dict = collections.OrderedDict()
for step_name, features in zip(steps_dict.iterkeys(), efel_features):
    features_dict[step_name] = {"soma":{}}
    for name, value in features.items():
        features_dict[step_name]["soma"][name] = [value, abs(value*0.05)]    
    
with open('./config/features.json', 'w') as fp:
    json.dump(features_dict, fp, indent = 4)

In [12]:
# Score a student's dictionary of parameters
students_params = {'me1': {u'ek.all': -107, u'e_pas.somatic': -63.3146, u'Ra.basal': 50.8833, 
               u'gbar_Kv3_1.axonal': 0.0100065, u'gbar_NaV.basal': 0.0205486, 
               u'gbar_Kv3_1.basal': 0.205458, u'e_pas.basal': -62.7085, 
               u'gbar_K_T.axonal': 0.004327727402343744, u'gamma_CaDynamics.somatic': 
               0.00169922, u'v_init': -81, u'cm.basal': 0.6721357746112606, 
               u'gbar_Ca_HVA.axonal': 3.83558e-05, u'gbar_NaV.somatic': 0.03811402789488886, 
               u'gbar_Kv2like.axonal': 0.0010615, u'gbar_Ca_LVA.somatic': 0.0060162, 
               u'gbar_Im_v2.basal': 0.00949979, u'gbar_SK.somatic': 0.0001051137, 
               u'g_pas.basal': 1.10433e-07, u'ena.all': 53.0, u'g_pas.axonal': 0.00200656, 
               u'gamma_CaDynamics.axonal': 0.0477399, u'gbar_SK.axonal': 0.00661877, 
               u'cm.axonal': 1.301052033032655, u'gbar_Ca_HVA.somatic': 5.9455e-05, 
               u'gbar_Kd.axonal': 0.00833495, u'gbar_Ih.basal': 0.0007, u'gbar_Ca_LVA.axonal': 
               0.00775489, u'Ra.somatic': 72.0757, u'e_pas.axonal': -71.1766, u'gbar_NaV.axonal': 
               0.08357652681547659, u'decay_CaDynamics.axonal': 811.497, u'cm.somatic': 
               7.9717441709009265, u'gbar_Ih.somatic': 3.46139e-06, 
               u'celsius': 34, u'Ra.axonal': 100.799, u'gbar_Kv3_1.somatic': 0.0021061531405053577, 
               u'decay_CaDynamics.somatic': 326.442, u'g_pas.somatic': 7.60303e-05},
'me2': {u'ek.all': -107, u'e_pas.somatic': -63.3146, u'Ra.basal': 50.8833, 
               u'gbar_Kv3_1.axonal': 0.0100065, u'gbar_NaV.basal': 0.0205486, 
               u'gbar_Kv3_1.basal': 0.205458, u'e_pas.basal': -62.7085, 
               u'gbar_K_T.axonal': 0.004327727402343744, u'gamma_CaDynamics.somatic': 
               0.00169922, u'v_init': -81, u'cm.basal': 0.6721357746112606, 
               u'gbar_Ca_HVA.axonal': 3.83558e-05, u'gbar_NaV.somatic': 0.03811402789488886, 
               u'gbar_Kv2like.axonal': 0.0010615, u'gbar_Ca_LVA.somatic': 0.0060162, 
               u'gbar_Im_v2.basal': 0.1, u'gbar_SK.somatic': 0.0001051137, 
               u'g_pas.basal': 1.10433e-07, u'ena.all': 53.0, u'g_pas.axonal': 0.00200656, 
               u'gamma_CaDynamics.axonal': 0.0477399, u'gbar_SK.axonal': 0.00661877, 
               u'cm.axonal': 1.301052033032655, u'gbar_Ca_HVA.somatic': 5.9455e-05, 
               u'gbar_Kd.axonal': 0.00833495, u'gbar_Ih.basal': 0.0007, u'gbar_Ca_LVA.axonal': 
               0.00775489, u'Ra.somatic': 72.0757, u'e_pas.axonal': -71.1766, u'gbar_NaV.axonal': 
               0.08357652681547659, u'decay_CaDynamics.axonal': 811.497, u'cm.somatic': 
               7.9717441709009265, u'gbar_Ih.somatic': 3.46139e-06, 
               u'celsius': 34, u'Ra.axonal': 100.799, u'gbar_Kv3_1.somatic': 0.0021061531405053577, 
               u'decay_CaDynamics.somatic': 326.442, u'g_pas.somatic': 7.60303e-05},
'me3': {u'ek.all': -107, u'e_pas.somatic': -63.3146, u'Ra.basal': 50.8833, 
               u'gbar_Kv3_1.axonal': 0.0100065, u'gbar_NaV.basal': 0.0205486, 
               u'gbar_Kv3_1.basal': 0.205458, u'e_pas.basal': -62.7085, 
               u'gbar_K_T.axonal': 0.004327727402343744, u'gamma_CaDynamics.somatic': 
               0.00169922, u'v_init': -81, u'cm.basal': 0.6721357746112606, 
               u'gbar_Ca_HVA.axonal': 3.83558e-05, u'gbar_NaV.somatic': 0.03811402789488886, 
               u'gbar_Kv2like.axonal': 0.0010615, u'gbar_Ca_LVA.somatic': 0.0060162, 
               u'gbar_Im_v2.basal': 0.3, u'gbar_SK.somatic': 0.0001051137, 
               u'g_pas.basal': 1.10433e-07, u'ena.all': 53.0, u'g_pas.axonal': 0.00200656, 
               u'gamma_CaDynamics.axonal': 0.0477399, u'gbar_SK.axonal': 0.00661877, 
               u'cm.axonal': 1.301052033032655, u'gbar_Ca_HVA.somatic': 5.9455e-05, 
               u'gbar_Kd.axonal': 0.00833495, u'gbar_Ih.basal': 0.0007, u'gbar_Ca_LVA.axonal': 
               0.00775489, u'Ra.somatic': 72.0757, u'e_pas.axonal': -71.1766, u'gbar_NaV.axonal': 
               0.08357652681547659, u'decay_CaDynamics.axonal': 811.497, u'cm.somatic': 
               7.9717441709009265, u'gbar_Ih.somatic': 3.46139e-06, 
               u'celsius': 34, u'Ra.axonal': 100.799, u'gbar_Kv3_1.somatic': 0.0021061531405053577, 
               u'decay_CaDynamics.somatic': 326.442, u'g_pas.somatic': 7.60303e-05},
'me4': {u'ek.all': -107, u'e_pas.somatic': -63.3146, u'Ra.basal': 50.8833, 
               u'gbar_Kv3_1.axonal': 0.0100065, u'gbar_NaV.basal': 0.1, 
               u'gbar_Kv3_1.basal': 0.205458, u'e_pas.basal': -62.7085, 
               u'gbar_K_T.axonal': 0.004327727402343744, u'gamma_CaDynamics.somatic': 
               0.00169922, u'v_init': -81, u'cm.basal': 0.6721357746112606, 
               u'gbar_Ca_HVA.axonal': 3.83558e-05, u'gbar_NaV.somatic': 0.03811402789488886, 
               u'gbar_Kv2like.axonal': 0.0010615, u'gbar_Ca_LVA.somatic': 0.0060162, 
               u'gbar_Im_v2.basal': 0.3, u'gbar_SK.somatic': 0.0001051137, 
               u'g_pas.basal': 1.10433e-07, u'ena.all': 53.0, u'g_pas.axonal': 0.00200656, 
               u'gamma_CaDynamics.axonal': 0.0477399, u'gbar_SK.axonal': 0.00661877, 
               u'cm.axonal': 1.301052033032655, u'gbar_Ca_HVA.somatic': 5.9455e-05, 
               u'gbar_Kd.axonal': 0.00833495, u'gbar_Ih.basal': 0.0007, u'gbar_Ca_LVA.axonal': 
               0.00775489, u'Ra.somatic': 72.0757, u'e_pas.axonal': -71.1766, u'gbar_NaV.axonal': 
               0.08357652681547659, u'decay_CaDynamics.axonal': 811.497, u'cm.somatic': 
               7.9717441709009265, u'gbar_Ih.somatic': 3.46139e-06, 
               u'celsius': 34, u'Ra.axonal': 100.799, u'gbar_Kv3_1.somatic': 0.0021061531405053577, 
               u'decay_CaDynamics.somatic': 326.442, u'g_pas.somatic': 7.60303e-05}}

results_dict = {} 
def score_all(all_params_dict):
    count = 0
    for name, params in all_params_dict.items():
        param_file = 'students_par/parameters_'+str(name)+'.json'
        with open(param_file, 'w+') as fp:
            json.dump(params, fp)
        
        param_configs = json.load(open(param_file))
        parameters = cell_model.define_parameters()

        morphology_file = 'Sst-IRES-Cre_Ai14_IVSCC_-183332.05.02.01_486041253_m.swc'
        morphology = ephys.morphologies.NrnFileMorphology(morphology_file, do_replace_axon=True)

        # Define mechanisms
        mechanisms = cell_model.define_mechanisms()

        # Create cell model
        abi_cell = ephys.models.CellModel('abi_cell', morph=morphology, mechs=mechanisms, params=parameters)

        # Define protocols
        proto_configs = json.load(open('config/protocols.json'))
        fitness_protocols = cell_evaluator.define_protocols()

        # Set-up a simulation and a fitness calculator
        sim = ephys.simulators.NrnSimulator()
        fitness_calculator = cell_evaluator.define_fitness_calculator(fitness_protocols)

        abi_cell.unfreeze(abi_cell.params)
        evaluator = ephys.evaluators.CellEvaluator(                                          
                cell_model=abi_cell,                                                       
                param_names=[],                                                    
                fitness_protocols=fitness_protocols,                                        
                fitness_calculator=fitness_calculator,                                      
                sim=sim)  

        student_responses = evaluator.run_protocols(protocols=fitness_protocols.values(), param_values = params)

        scores =  fitness_calculator.calculate_scores(student_responses)
        sum = numpy.sum([value for value in scores.values()])
        avg = numpy.mean([value for value in scores.values()])

        print "\nSum of all feature values: " + str(sum)
        print "Average of the feature values: " + str(avg)

        results_dict[name] = [sum, avg]
        
        count+=1
        
        print "Evaluated %d of %d students." % (count, len(students_params))
        
score_all(students_params)


Sum of all feature values: 523.840031111
Average of the feature values: 130.960007778
Evaluated 1 of 4 students.

Sum of all feature values: 23.4759148057
Average of the feature values: 5.86897870144
Evaluated 2 of 4 students.

Sum of all feature values: 523.873707566
Average of the feature values: 130.968426891
Evaluated 3 of 4 students.

Sum of all feature values: 520.352684096
Average of the feature values: 130.088171024
Evaluated 4 of 4 students.


In [32]:
sorted_results = sorted(results_dict.items(), key=lambda i: i[1][0])

print "\n The winner is ... \n.......................................\n.......................................\n......................................."


 The winner is ... 
.......................................
.......................................
.......................................


In [40]:
print "\n"+str(sorted_results[0][0])

print "\nWith a final error of %d." % sorted_results[0][1][0]


me1

With a final error of 23.


In [45]:
print "\nAll scores...\n"
for result in sorted_results:
    print result[0], result[1][0]


All scores...

me1 23.4759148057
me2 520.352684096
me4 523.840031111
me3 523.873707566
